# Subpixellic Methods for Sidelobes Suppression and Strong Targets: TESTS

In [ ]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from PIL import Image, ImageSequence
import rasterio
import mvalab
from numpy.fft import fft, ifft, fft2, ifft2, fftshift, ifftshift
import nfft
import plotly.express as px
from irregular_translations import *
from regularization import *

### Opening the data

In [ ]:
#multichannel TIF
#Channel 0 = partie reelle
#Channel 1 = partie imaginaire
u = Image.open('u0.tif')
im_li = []

for i, page in enumerate(ImageSequence.Iterator(u)):
    im_li.append(np.array(page))
im = im_li[0] + 1j * im_li[1]
mvalab.visusar(im, 3)

In [ ]:
#Working on a smaller image
test_im = im[580:780, 565:765] 
print(test_im.shape)
mvalab.visusar(test_im, 3)

In [ ]:
#Zoom to see a target and the sinusc signature
test_im_zoom = im[650:710, 635:695]
#mvalab.visusar(test_im[20:55, 30:,], 3)
mvalab.visusar(test_im_zoom, 3)

#### Loading the resampled image v

In [ ]:
v = Image.open('v0.tif')
im_li = []
for i, page in enumerate(ImageSequence.Iterator(v)):
    im_li.append(np.array(page))
im_resampled = im_li[0] + 1j * im_li[1]

#Working on the same smaller image
v = im_resampled[580:780, 565:765] 
mvalab.visusar(v, 3)

## Finding the translation maps

### Test for vertical translations

In [ ]:
#Considering we only have one target for this example
#Get the index of the maximum amplitude ie the target
x_theo, y_theo = np.unravel_index(np.argmax(np.abs(test_im_zoom), axis=None), test_im_zoom.shape)

im_t, t_li = translate(test_im_zoom, 20, 1)
for i in range(len(im_t)):
    plt.figure()
    plt.imshow(im_t[i][35:55, 30:,], cmap='gray')
    plt.title('Translation  = %f' %i)

### TV minimization

In [ ]:
#Evolution of the TV for a localized target
displacement, history_x, history_y = find_displacement(test_im_zoom, 20, 5, tv_history=(x_theo, y_theo))
plt.figure()
plt.plot(history_x, c='green')
plt.title('Evolution of TV wrt translation X (localization=target)')
plt.xlabel('Translation over X (horizontal)')
plt.ylabel('TV')


plt.figure()
plt.plot(history_y, c='red')
plt.title('Evolution of TV wrt translation Y (localization=target)')
plt.xlabel('Translation over Y (vertical)')
plt.ylabel('TV')


In [ ]:
#Evolution of the TV for a pixel in a speckle area
displacementS, history_xS, history_yS = find_displacement(test_im_zoom, 20, 5, tv_history=(30, 30))
plt.figure()
plt.plot(history_x, c='cyan')
plt.title('Evolution of TV wrt translation X (localization=speckle)')
plt.xlabel('Translation over X (horizontal)')
plt.ylabel('TV')

plt.figure()
plt.plot(history_y, c='magenta')
plt.title('Evolution of TV wrt translation Y (localization=speckle)')
plt.xlabel('Translation over Y (vertical)')
plt.ylabel('TV')


In [ ]:
#Comparison of the scales for a target and a speckle pixel
legend = ['target', 'speckle']
plt.figure()
plt.plot(history_y, c='red')
plt.plot(history_yS, c='magenta')
plt.title('Evolution of TV wrt translation Y')
plt.xlabel('Translation over Y (vertical)')
plt.ylabel('TV')


## Regularization of the Translation maps using Chambolle-Pock algorithm

### TV amplitude based Weights

In [ ]:
displacement, Wx, Wy = find_displacement(test_im, 20, 5, ampli_weights=True)
Tx, Ty = displacement[:,:,0], displacement[:,:,1]

In [ ]:
#Regularization for Tx
Lambda = 0.1
Tx_reg = chambolle_pock(Wx*Tx, Wx, Lambda)

#Visualization
#Original image
mvalab.visusar(test_im, 3)

#Original translation map
fig = px.imshow(Tx, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Original translation map Tx")
fig.show()

#Weights
fig = px.imshow(Wx, color_continuous_scale=px.colors.sequential.algae)
fig.update_layout(title="Tx Weights based on TV amplitude")
fig.show()

#Regularized translation map
fig = px.imshow(Tx_reg, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Regularized translation map Tx_reg")
fig.show()


In [ ]:
#Regularization for Ty
Lambda = 0.1
Ty_reg = chambolle_pock(Wy*Ty, Wy, Lambda)

#Visualization
#Original image
mvalab.visusar(test_im, 3)

#Original translation map
fig = px.imshow(Ty, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Original translation map Ty")
fig.show()

#Weights
fig = px.imshow(Wy, color_continuous_scale=px.colors.sequential.algae)
fig.update_layout(title="Ty Weights based on TV amplitude")
fig.show()

#Regularized translation map
fig = px.imshow(Ty_reg, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Regularized translation map Ty_reg")
fig.show()


In [ ]:
#Quick study of the influence of the regularization parameter Lambda
#Test on Tx
Lambda_li = np.linspace(0, 10, num=50)
for l in Lambda_li:
    print('Regularization parameter: lambda=', l)
    Tx_reg = chambolle_pock(Wx*Tx, Wx, l)
    fig = px.imshow(Tx_reg, color_continuous_scale=px.colors.sequential.Cividis_r)
    fig.update_layout(title="Regularized translation map Tx_reg")
    fig.show()

### R Weights

In [ ]:
Rx, Ry = compute_R(v)

#### Quick study of the importance of the parameter omega

In [ ]:
omega_li = np.arange(0, 100, 5)
for omega in omega_li:
    print('OMEGA = ', omega)
    Rx, Ry = compute_R(v, omega=omega)
    fig = px.imshow(Rx, color_continuous_scale=px.colors.sequential.Burg_r)
    fig.update_layout(title="Rx")
    fig.show()

#### Having a look at the distribution of Rx and Ry

In [ ]:
plt.figure()
plt.hist(Rx.reshape(-1), bins=100, density=True)
plt.title('Histogram of Rx : Rayleigh distribution')

plt.figure()
cumhist_x, bins_x, _ = plt.hist(Rx.reshape(-1), bins=100, density=True, cumulative=True)
plt.title('Cumulative histogram of Rx')


plt.figure()
plt.hist(Ry.reshape(-1), bins=100, density=True)
plt.title('Histogram of Ry : Rayleigh distribution')

plt.figure()
cumhist_y, bins_y, _ = plt.hist(Ry.reshape(-1), bins=100, density=True, cumulative=True)
plt.title('Cumulative histogram of Ry')

#### Thresholding

In [ ]:
#Thresholding for Rx and Ry
#Keep only the point above in both cases
target_x = thresholding5(Rx, cumhist_x, bins_x)
target_y = thresholding5(Ry, cumhist_y, bins_y)
target = target_x*target_y
R = Rx + Ry
W = get_R_weights(R, target)

In [ ]:
#Regularization for Tx
Lambda = 0.1
Tx_reg = chambolle_pock(W*Tx, W, Lambda)

#Visualization
#Original image
mvalab.visusar(test_im, 3)

#Original translation map
fig = px.imshow(Tx, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Original translation map Tx")
fig.show()

#Weights
fig = px.imshow(W, color_continuous_scale=px.colors.sequential.algae)
fig.update_layout(title="Tx Weights based on R")
fig.show()

#Regularized translation map
fig = px.imshow(Tx_reg, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Regularized translation map Tx_reg")
fig.show()


In [ ]:
#Regularization for Ty
Lambda = 0.1
Ty_reg = chambolle_pock(W*Ty, W, Lambda)

#Visualization
#Original image
mvalab.visusar(test_im, 3)

#Original translation map
fig = px.imshow(Ty, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Original translation map Ty")
fig.show()

#Weights
fig = px.imshow(W, color_continuous_scale=px.colors.sequential.algae)
fig.update_layout(title="Ty Weights based on R")
fig.show()

#Regularized translation map
fig = px.imshow(Ty_reg, color_continuous_scale=px.colors.sequential.Cividis_r)
fig.update_layout(title="Regularized translation map Ty_reg")
fig.show()
